In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch

In [2]:
#create custom data class

class CustomDataset(Dataset):
    def __init__(self,low_idx,high_idx):
        self.data = pd.read_csv("./dataset/IRIS.csv", sep=",", skiprows=range(low_idx,high_idx), header=1)
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sepal_length = torch.tensor(self.data.iloc[idx, 0])
        sepal_width = self.data.iloc[idx, 1]
        petal_length = self.data.iloc[idx, 2]
        petal_width = self.data.iloc[idx, 3] 
        label = self.data.iloc[idx, 4]
        
        tensor_data = torch.tensor([sepal_length,sepal_width,sepal_width,sepal_width],dtype=torch.float32)
        
        return tensor_data , label

In [3]:
#just my afwull way to seperate training and testing dataset
# 80% training , 20% Test

train_dataset = CustomDataset(120,149)
test_dataset = CustomDataset(1,120)
len(train_dataset), len(test_dataset)


(120, 30)

In [4]:
#create train_dataloader and test_dataloader
# 80% for training, 20% of data for testing -> 149*0.8 = 119.2 -> 120 , 29
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True) 
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [5]:
#4 inputs and 3  outputs 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack =  nn.Sequential(
         nn.Linear(4,128),
         nn.ReLU(),
         nn.Linear(128,64),
         nn.ReLU(),
         nn.Linear(64,3),
         )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits

In [6]:
#map labels into tensor of shape [1, 0, 0]
def transform_label(label_data):
    data = []
    for i in label_data:
        if i == "Iris-setosa":
            data.append(torch.tensor([1,0,0], dtype=torch.float32))
        if i == "Iris-versicolor": 
            data.append(torch.tensor([0,1,0], dtype=torch.float32))
        if i == "Iris-virginica":
            data.append(torch.tensor([0,0,1], dtype=torch.float32))
    return torch.stack(data)

In [7]:
X_train, train_labels = next(iter(train_loader))
#train_labels before mapping tesnors 
train_labels

('Iris-setosa',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-setosa',
 'Iris-versicolor')

In [8]:
#train_labels, X_train = next(iter(train_loader))
train_labels = transform_label(train_labels)
X_train.shape, train_labels.shape, X_train.dtype, train_labels.dtype

(torch.Size([10, 4]), torch.Size([10, 3]), torch.float32, torch.float32)

In [9]:
X_train, train_labels

(tensor([[4.8000, 3.0000, 3.0000, 3.0000],
         [6.0000, 2.7000, 2.7000, 2.7000],
         [4.8000, 3.1000, 3.1000, 3.1000],
         [5.2000, 3.4000, 3.4000, 3.4000],
         [5.7000, 2.8000, 2.8000, 2.8000],
         [5.9000, 3.2000, 3.2000, 3.2000],
         [6.4000, 3.2000, 3.2000, 3.2000],
         [6.3000, 2.9000, 2.9000, 2.9000],
         [5.4000, 3.9000, 3.9000, 3.9000],
         [5.5000, 2.6000, 2.6000, 2.6000]]),
 tensor([[1., 0., 0.],
         [0., 1., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 0., 1.],
         [1., 0., 0.],
         [0., 1., 0.]]))

In [10]:
#create model
lr = 0.01
model = NeuralNetwork()
optim = torch.optim.Adam(model.parameters(), lr=lr)
loss = torch.nn.CrossEntropyLoss()
#loss = torch.nn.MSELoss()
#Weights are by default torch.32 not 64 --> error message 

In [11]:
n_iters = 1000
for epochs in range(n_iters):
    #forward pass and loss
    #test_labels, X_test = next(iter(test_loader))
    if epochs%100 == 0:
        print(f"\n epochs: {epochs+100}/{n_iters} \n")
    for i,(inputs, labels) in enumerate(train_loader):
        out = model(inputs)
        train_labels = transform_label(labels)
        l = loss(out, train_labels)
        #backward pass
        l.backward()
        #update weights
        optim.step()
        optim.zero_grad()
        
        if epochs%100 == 0:
            print(f"loss: {l.item()}")
        #if i % 1 == 0:
            #print(f" steps: {i+1}, loss : {l.item()}")


 epochs: 100/1000 

loss: 0.9915851354598999
loss: 1.2492998838424683
loss: 1.5773557424545288
loss: 1.4274284839630127
loss: 1.0542516708374023
loss: 1.0552093982696533
loss: 1.0616728067398071
loss: 0.9820119738578796
loss: 1.1465730667114258
loss: 0.8269017338752747
loss: 1.177398920059204
loss: 1.0440948009490967

 epochs: 200/1000 

loss: 0.5441662073135376
loss: 0.27309730648994446
loss: 0.44869765639305115
loss: 0.2771966755390167
loss: 0.19319379329681396
loss: 0.4082963466644287
loss: 0.36315280199050903
loss: 0.4315318167209625
loss: 0.49728164076805115
loss: 0.27391141653060913
loss: 0.1712169647216797
loss: 0.3732554316520691

 epochs: 300/1000 

loss: 0.2363680601119995
loss: 0.2217107117176056
loss: 0.26337116956710815
loss: 0.2428346425294876
loss: 0.49308425188064575
loss: 0.5259215235710144
loss: 0.4802197813987732
loss: 0.608556866645813
loss: 0.13630448281764984
loss: 0.24147415161132812
loss: 0.101205013692379
loss: 0.28111451864242554

 epochs: 400/1000 

loss: 0.

In [12]:
#X_test, test_labels = next(iter(test_loader))
#test_labels = transform_label(test_labels)
#x_label_predicted = model(X_test)
#x_label_predicted, test_labels

In [13]:
#_, x_label_predicted_hat = torch.max(x_label_predicted,1)
#x_label_predicted_hat

In [14]:
def final_x(x_hat):
    list_tensors = []
    for i in x_hat:
        print(i)
        final_tensor = torch.zeros(1,3)
        final_tensor[0:,i] = 1
        list_tensors.append(final_tensor)
    return torch.cat(list_tensors)
    
    # tensor[0:,1] = 4 to accses specific value 

In [15]:
#final_x = final_x(x_label_predicted_hat)

In [16]:
def get_accuracy(model_prediction, test_labels):
    idx = 0
    number_pred = 0
    counter = 0
    while idx < 10:
        print(f" \n Tensor[{idx+1}]")
        for i in range(3):
            print(f"model: {model_prediction[idx][i].item()}, label: {test_labels[idx][i].item()}")
            if model_prediction[idx][i].item() == test_labels[idx][i].item():

                counter += 1
                if counter == 3:
                    number_pred += 1
        counter = 0
          
        idx +=1
    accuracy = number_pred/10 *100
    return print(f"\naccuracy of model {number_pred/10 *100}%")

In [17]:
#get_accuracy(final_x, test_labels)

In [21]:
#test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)
import accuracy_model as ac
test_model_acc = ac.data_loader
test_model_acc.accuracy(model, test_loader, batch_size=10)

accuracy per batch 0:
40.0%
accuracy per batch 1:
50.0%
accuracy per batch 2:
40.0%

total accuracy of model 43.33%


In [19]:
# save model 
#torch.save(model.state_dict(), "./model")